In [5]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os, re, math
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
inputs_path = os.getenv('J111_INPUTS')
outputs_path = os.getenv('J111_OUTPUTS')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Set path to the input excel workbooks
da_xlsx = os.path.join(inputs_path, 'INPUT_DevelopmentApplications.xlsx')
proj_xlsx = os.path.join(inputs_path, 'INPUT_InterimProjections_Residential.xlsx')

# Get full path of Input GDBs
pscap_gdb_path = os.path.join(spatial_path, 'redland_gam_ps_capacity.gdb')
curdev_gdb_path = os.path.join(spatial_path, 'redland_gam_current_dev.gdb')
alloc_gdb_path = os.path.join(spatial_path, 'redland_gam_alloc.gdb')

# Set arcpy workspace
arcpy.env.workspace = os.path.join(env_path, 'J111_redland_gam.gdb')

# *Variables*

In [6]:
''' 
----------------------------------
Define allocation variables
----------------------------------
'''

years = [2026, 2031, 2036, 2041, 2046, 2051]
res_cols = ['det_dwl', 'att_dwl']

# *Development Assessment Data*

In [7]:
''' 
----------------------------------
Import development assessment data
----------------------------------
'''

# Import development assessment data table
da_data = pd.read_excel(da_xlsx, 'DevelopmentApplications').drop(['app_detail', 'address', 'decision'], axis=1)

# Inspect
da_data.head()

,appid,landnos,decision_date,att_dwl,det_dwl,commercial,community,education,health,industry,retail,other
0,MCU013891,115384;115322,43115,12,0,0,0,0,0,0,0,0
1,MCU17/0111,118178;118352;118341;118185,43159,72,0,0,0,0,0,0,0,0
2,MCU18/0037,995322,43171,14,0,0,0,0,0,0,0,0
3,MCU013865,132331,43173,0,0,0,185,0,0,0,0,720
4,MCU18/0001,314780,43214,0,0,0,0,0,0,0,0,0


In [8]:
''' 
----------------------------------
Import and prepare the parcels to council land relational dataframe
----------------------------------
'''

# Import the Council Property to DCDB Base lots relational list
parcels_to_land = pd.DataFrame.spatial.from_table(os.path.join(curdev_gdb_path, 'INPUT_TB_DCDB_CouncilPropertyRel')).drop(['OBJECTID'], axis=1)

# Select only the required column
parcels_land = parcels_to_land.loc[:, ('segpar', 'landnos')]

# Split the land numbers string into list
parcels_land.landnos = parcels_land.landnos.apply(lambda v: [int(x) for x in v.split(';')])

# Explode on the land numbers column (this will create duplicate parcel segpar records)
parcels_land = parcels_land.explode('landnos', ignore_index=True).rename({'landnos' : 'landno'}, axis=1)

# Inspect
parcels_land.head()

,segpar,landno
0,28663092,146617
1,28460061,996564
2,32043083,145846
3,20484023,144907
4,20689095,112272


In [9]:
''' 
----------------------------------
Relate DA data to land parcels
----------------------------------
'''

# Get subset of the DA data
da_land = da_data.loc[:, ('appid', 'landnos')]

# Set and explode list on the landnos
da_land['landnos'] = da_land['landnos'].astype(str) + ';'
da_land['landnos'] = da_land['landnos'].str.split(';')
da_land['landnos'] = da_land.apply(lambda row: list(filter(None, row.landnos)), axis=1)
da_land = da_land.explode('landnos').rename({'landnos' : 'landno'}, axis=1)
da_land['landno'] = da_land.apply(lambda row: int(str(row.landno)), axis=1)
da_land['landno'] = da_land['landno'].astype('Int64')

# Get the segpar for each landno
da_land = pd.merge(da_land, parcels_land, on='landno', how='left')

# Import the aggregate details table from geodatabase
agg_details = pd.DataFrame.spatial.from_table(os.path.join(curdev_gdb_path, 'GEN_TB_DCDB_AggregateDetails')).drop(['OBJECTID'], axis=1)

# Drop duplicates from the aggregate details
agg_details = agg_details.drop(['aggtype', 'segpar'], axis=1).drop_duplicates().sort_values(by='aggid').reset_index(drop=True)

# Join aggid to DA data via landno
da_land = pd.merge(da_land, agg_details, on='landno', how='left')

# Split DA data into those with an aggid and those without
da_land_agg = da_land.query("aggid == aggid").drop(['landno', 'segpar'], axis=1).drop_duplicates().reset_index(drop=True)
da_land_other = da_land.query("aggid != aggid").drop(['landno', 'aggid'], axis=1).drop_duplicates().reset_index(drop=True)

# Inspect
da_land_agg.head()

,appid,aggid
0,MCU18/0037,AGG000244
1,MCU17/0053,AGG000832
2,MCU18/0043,AGG000312
3,MCU17/0053.01,AGG000832
4,MCU18/0220,AGG002528


In [10]:
# Import propbase
propbase = pd.DataFrame.spatial.from_featureclass(os.path.join(curdev_gdb_path, 'GEN_FC_PropertyBase')).drop(['OBJECTID', 'SHAPE'], axis=1)

# Inspect
propbase.head()

,pbno,segpar,aggid,sa2_name
0,8000000,NaN,AGG000001,Cleveland
1,8000001,NaN,AGG000002,Cleveland
2,8000002,NaN,AGG000003,Redland Islands
3,8000003,NaN,AGG000004,Cleveland
4,8000004,NaN,AGG000005,Capalaba


In [12]:
''' 
----------------------------------
Relate DA data to propbase
----------------------------------
'''

# Split the propbase into aggregates and otherwise
pb_agg = propbase.query("aggid == aggid").drop(['segpar', 'sa2_name'], axis=1).drop_duplicates().reset_index(drop=True)
pb_other = propbase.query("aggid != aggid").drop(['aggid', 'sa2_name'], axis=1).drop_duplicates().reset_index(drop=True)
pb_other['segpar'] = pb_other['segpar'].astype('Int64')

# Join the propbase data onto the respective dataframes and concatenate
pb_da = pd.concat([
    pd.merge(da_land_agg, pb_agg, on='aggid', how='left').drop(['aggid'], axis=1),
    pd.merge(da_land_other, pb_other, on='segpar', how='left').drop(['segpar'], axis=1)
]).sort_values('pbno').reset_index(drop=True)

# Join DA data back onto related table
pb_da = pd.merge(pb_da, da_data.drop(['landnos'], axis=1), on='appid', how='left')

# ----------------------------------
# IMPORTANT !!!

# If there are multiple applications on a property take the most recent
pb_da = pb_da.sort_values('decision_date').drop_duplicates('pbno', keep='last').drop(['decision_date'], axis=1).reset_index(drop=True)

# ----------------------------------

# Create a subset of the table and allocate development data where situated across multiple properties
pb_da_sub = pb_da.drop(['pbno'], axis=1)
pb_da_sub['props'] = 1
ps_da_dub_g1 = pb_da_sub.loc[:, ('appid', 'props')].groupby('appid').agg('count').reset_index()
ps_da_dub_g2 = pb_da_sub.groupby('appid').agg('max').drop(['props'], axis=1).reset_index()
pb_da_sub = pd.merge(ps_da_dub_g1, ps_da_dub_g2, on='appid', how='left')

# Force all values on development columns as floats
pb_da_sub.iloc[:,2:] = pb_da_sub.iloc[:,2:].astype(float)

# Divide the development columns by the number to props to allocate
pb_da_sub.iloc[:,2:] = pb_da_sub.iloc[:,2:].div(pb_da_sub.props, axis=0)

# Drop columns
pb_da_sub = pb_da_sub.drop(['props'], axis=1)

# ----------------------------------

# Merge cleaned data back onto the probase-da table
pb_da = pd.merge(pb_da.iloc[:,:2], pb_da_sub, on='appid', how='left')

# Split the residential and non-residential components into separate dataframes
pb_da_res = pb_da.drop([col for col in pb_da.columns if col not in ['app_id', 'pbno', 'att_dwl', 'det_dwl']], axis=1)

# Inspect
pb_da_res.head()

,pbno,att_dwl,det_dwl
0,8056279,6.0,0.0
1,8023850,6.0,0.0
2,8000243,14.0,0.0
3,8019617,0.0,0.0
4,8049235,0.0,0.0


# *Current Development Statistics*

In [13]:
''' 
----------------------------------
Import and prepare current development stats
----------------------------------
'''

# Import current development metrics from geodatabase
pb_curdev_lyr = pd.DataFrame.spatial.from_featureclass(os.path.join(curdev_gdb_path, 'OUT_FC_PropertyBase_CurrentDev')).drop(['OBJECTID'], axis=1)

# Make a copy
pb_curdev = pb_curdev_lyr.drop(['SHAPE', 'prop_use'], axis=1)

# Add the long term accommodation calcs to attached dwellings and drop column
pb_curdev['att_dwl'] = pb_curdev['att_dwl'] + pb_curdev['accom_long']
pb_curdev = pb_curdev.drop(['accom_long'], axis=1)

# Re-name columns to identify as 2021 
pb_curdev = pb_curdev.rename({col: col + '_2021' for col in pb_curdev.columns if col not in ['pbno', 'sa2_name']}, axis=1)

# Split the residential and non-residential components into separate dataframes
pb_curdev_res = pb_curdev.drop([col for col in pb_curdev.columns if col not in ['pbno', 'sa2_name', 'det_dwl_2021', 'att_dwl_2021']], axis=1).reset_index(drop=True)

# Inspect
pb_curdev_res.head()

,pbno,sa2_name,det_dwl_2021,att_dwl_2021
0,8000000,Cleveland,0.0,9.0
1,8000001,Cleveland,0.0,0.0
2,8000002,Redland Islands,0.0,18.0
3,8000003,Cleveland,0.0,146.0
4,8000004,Capalaba,0.0,0.0


# *Development Capacity Statistics*

In [14]:
''' 
----------------------------------
Import and prepare development capacity table
----------------------------------
'''

# Import development table from geodatabase
pb_devcap_lyr = pd.DataFrame.spatial.from_featureclass(os.path.join(pscap_gdb_path, 'OUT_FC_PropertyBase_DevCapacity')).drop(['OBJECTID'], axis=1)

# Copy the layer into a dataframe
pb_devcap = pb_devcap_lyr.copy().drop(['zone', 'property_area', 'eda', 'SHAPE'], axis=1)

# Re-name columns to identify as planned capacity
pb_devcap = pb_devcap.rename({col: col + '_cap' for col in pb_devcap.columns if col not in ['pbno', 'sa2_name', 'res_conv', 'nonres_conv']}, axis=1)

# Select only those columns that will convert
pb_devcap['conv'] = pb_devcap['res_conv'] + pb_devcap['nonres_conv']
pb_devcap = pb_devcap.query("conv > 0").drop(['conv'], axis=1)

# Get residential statistics
pb_devcap_res = pb_devcap.query("res_conv > 0").drop([col for col in pb_devcap.columns if col not in ['pbno', 'sa2_name', 'res_conv', 'det_dwl_cap', 'att_dwl_cap']], axis=1).reset_index(drop=True)

# Inspect
pb_devcap_res.head()

,pbno,sa2_name,res_conv,det_dwl_cap,att_dwl_cap
0,8000000,Cleveland,1,0.0,13.0
1,8000007,Victoria Point,1,0.0,12.0
2,8000009,Capalaba,1,0.0,102.0
3,8000014,Wellington Point,1,0.0,5.0
4,8000019,Cleveland,1,0.0,6.0


# *Allocation functions*

In [15]:
# Helper function for initializing spare capacity
def InitSpareCapacity(prop, dev_type, proj_year):

    dev_year = prop[f'{dev_type}{proj_year}']
    dev_cap = prop[f'{dev_type}cap']

    if dev_year and dev_cap > 0:
        if dev_cap - dev_year > 0:
            return dev_cap - dev_year
        else:
            return 0
    else:
        return dev_cap

# Helper function for setting remaining dev values in projection year (accounting for existing data)
def SetYear(prop, dev_type, proj_year, base_year):

    dev_year = prop[f'{dev_type}{proj_year}']
    dev_prev = prop[f'{dev_type}{base_year}']

    if dev_year == 0:
        return dev_prev
    else:
        return dev_year

# *Residential growth allocation process*

In [16]:
''' 
----------------------------------
Handle DA residential allocation in the 2026 horizon
----------------------------------
'''

# Prepare initial dataframe
pb_alloc_res_da = pb_curdev_res.set_index('pbno')

# Set new columns to 0
for col in res_cols:
    pb_alloc_res_da[f'{col}_2026'] = 0.0
    pb_alloc_res_da[f'{col}_cap'] = 0.0

pb_alloc_res_da['res_conv'] = 0

# Set 2026 columns with da data where matched on pbno
pb_alloc_res_da.update(pb_da_res.rename({col: col + '_2026' for col in pb_da_res.columns if col != 'pbno'}, axis=1).set_index('pbno'))

# Set capacity columns where matched on pbno
pb_alloc_res_da.update(pb_devcap_res.drop(['sa2_name'], axis=1).set_index('pbno'))

# Update the dwelling numbers across all columns
for col in res_cols:
    
    # If there is da data for 2026 set this to be the capacity for these site
    pb_alloc_res_da[f'{col}_cap'] = pb_alloc_res_da.apply(lambda row: row[f'{col}_2026'] if row[f'{col}_2026'] > 0 else row[f'{col}_cap'],axis=1)

# Inspect
pb_alloc_res_da.sort_values('res_conv', ascending=False).head()

,sa2_name,det_dwl_2021,att_dwl_2021,det_dwl_2026,det_dwl_cap,att_dwl_2026,att_dwl_cap,res_conv
pbno,,,,,,,,
8000000,Cleveland,0.0,9.0,0.0,0.0,0.0,13.0,1.0
8041325,Redland Bay,1.0,0.0,0.0,2.0,0.0,0.0,1.0
8029094,Redland Islands,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8007466,Thorneside,1.0,0.0,0.0,0.0,0.0,4.0,1.0
8029097,Redland Islands,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [18]:
''' 
----------------------------------
Import QGSO residential projections
----------------------------------
'''

# Import detached dwelling projections
det_proj = pd.read_excel(proj_xlsx, 'OUT_DetDwelProj')

# Import attached dwelling projections
att_proj = pd.read_excel(proj_xlsx, 'OUT_AttDwelProj')

# Select only the growth columns
det_proj = det_proj.loc[:, ~det_proj.columns.str.startswith('proj_')]
att_proj = att_proj.loc[:, ~att_proj.columns.str.startswith('proj_')]

# Rename the growth columns to type
det_proj = det_proj.rename({col: 'det_dwl_' + col.lstrip('growth_') for col in det_proj.columns if col != 'sa2_name'}, axis=1)
att_proj = att_proj.rename({col: 'att_dwl_' + col.lstrip('growth_') for col in att_proj.columns if col != 'sa2_name'}, axis=1)

# Combine tables
res_proj = pd.concat([det_proj, att_proj.drop(['sa2_name'], axis=1)], axis=1).set_index('sa2_name')

# Inspect
res_proj

,det_dwl_2026,det_dwl_2031,det_dwl_2036,det_dwl_2041,det_dwl_2046,det_dwl_2051,att_dwl_2026,att_dwl_2031,att_dwl_2036,att_dwl_2041,att_dwl_2046,att_dwl_2051
sa2_name,,,,,,,,,,,,
Alexandra Hills,77,65,47,48,15,15,1,21,15,15,5,5
Birkdale,192,90,147,147,32,32,77,75,30,30,13,13
Capalaba,102,97,197,195,36,36,431,409,50,50,59,60
Cleveland,205,103,1,1,27,27,1539,1432,1151,1149,411,431
Ormiston,111,80,81,63,20,20,74,66,1,1,8,8
Redland Bay,606,727,672,863,171,174,157,111,26,28,28,29
Redland Islands,595,637,473,469,143,145,0,0,0,0,0,0
Sheldon - Mount Cotton,13,32,26,5,5,5,1,1,1,1,1,1
Thorneside,87,4,1,1,9,9,7,0,0,0,1,1


In [19]:
''' 
----------------------------------
Correct the residential growth projections to account for the already allocated DA data
----------------------------------
'''

# Summarise the DA data by SA2
da_res_sum = pb_alloc_res_da.reset_index(drop=True).loc[:, ('sa2_name', 'det_dwl_2026', 'att_dwl_2026')].groupby('sa2_name').agg('sum')

# Get a copy of the residential projections dataframe
res_proj_corr = res_proj.copy()

# Get column unique names (remove years)
res_dev_cols = list(set([col[:-4] for col in res_proj.columns if col != 'sa2_name']))

# Move through the summarised DA table and correct the residential projections for each SA2 carrying over remainder between years
for col in res_dev_cols:
    
    carry = da_res_sum[f'{col}2026']

    for year in years:

        sub = res_proj_corr[f'{col}{year}'].sub(carry)
        pos = sub.apply(lambda x: 0 if x <= 0 else x)
        neg = sub.apply(lambda x: abs(x) if x < 0 else 0)
        res_proj_corr[f'{col}{year}'] = pos.apply(lambda x: 0 if x <= 0 else x)
        
        carry = neg

# Inspect
res_proj_corr

,det_dwl_2026,det_dwl_2031,det_dwl_2036,det_dwl_2041,det_dwl_2046,det_dwl_2051,att_dwl_2026,att_dwl_2031,att_dwl_2036,att_dwl_2041,att_dwl_2046,att_dwl_2051
sa2_name,,,,,,,,,,,,
Alexandra Hills,77.0,65,47,48,15,15,0.0,0.0,0.0,0.0,0.0,0.0
Birkdale,192.0,90,147,147,32,32,62.0,75.0,30.0,30.0,13.0,13.0
Capalaba,102.0,97,197,195,36,36,271.0,409.0,50.0,50.0,59.0,60.0
Cleveland,205.0,103,1,1,27,27,1087.0,1432.0,1151.0,1149.0,411.0,431.0
Ormiston,111.0,80,81,63,20,20,35.0,66.0,1.0,1.0,8.0,8.0
Redland Bay,606.0,727,672,863,171,174,93.0,111.0,26.0,28.0,28.0,29.0
Redland Islands,595.0,637,473,469,143,145,0.0,0.0,0.0,0.0,0.0,0.0
Sheldon - Mount Cotton,13.0,32,26,5,5,5,1.0,1.0,1.0,1.0,1.0,1.0
Thorneside,87.0,4,1,1,9,9,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
''' 
----------------------------------
Allocate residential growth projections per property based on remaining capacity
----------------------------------
'''

# Prepare initial dataframe
pb_alloc_res = pb_alloc_res_da.copy()

# Set spare capacity
for col in res_dev_cols:
    pb_alloc_res[f'{col}spare'] = pb_alloc_res.apply(lambda row: InitSpareCapacity(row, col, 2026), axis=1)

# Set first projeciton year (2026)
for col in res_dev_cols:
    pb_alloc_res[f'{col}{2026}'] = pb_alloc_res.apply(lambda row: SetYear(row, col, 2026, 2021), axis=1)

# Drop residential conversion column as this is now redundant
pb_alloc_res = pb_alloc_res.drop(['res_conv'], axis=1)

# ----------------------------------
# Set empty years

for col in res_dev_cols:

    for year in years[1:]:

        pb_alloc_res[f'{col}{year}'] = 0.0

# ----------------------------------
# Set column order
res_col_order = ['sa2_name'] + \
    [res_dev_cols[1] + str(year) for year in [2021] + years] + ['det_dwl_spare', 'det_dwl_cap'] + \
    [res_dev_cols[0] + str(year) for year in [2021] + years] + ['att_dwl_spare', 'att_dwl_cap']

pb_alloc_res = pb_alloc_res[res_col_order]

# ----------------------------------
# ALLOCATION LOOP

for col in res_dev_cols:

    for i, year in enumerate(years):

        # Set the residential development column names
        dev_year = f'{col}{year}'
        dev_prev = f'{col}{year - 5}'
        dev_cap = f'{col}cap'
        dev_spare = f'{col}spare'

        # Get the summary statistics for the allocation
        year_spare = pb_alloc_res.reset_index().loc[:, ('sa2_name', dev_spare)].groupby('sa2_name').sum().squeeze()
        year_proj = res_proj_corr.reset_index().loc[:, ('sa2_name', dev_year)].set_index('sa2_name').squeeze()

        # Calculate the properties contribution to the spare capacity withint the SA2 based on its development capacity
        pb_alloc_res['contrib'] = pb_alloc_res.apply(lambda row: row[dev_cap] / year_spare[f'{row.sa2_name}'] if row[dev_spare] > 0 else 0, axis=1)

        # For each statistic area grab only those properties with a contribution
        for sa2_name, proj in year_proj.iteritems():
            
            # Get subset of the allocation dataframe
            pb_alloc_sub = pb_alloc_res.query("sa2_name == @sa2_name").reset_index().loc[:, ('pbno', f'{dev_prev}', f'{dev_year}', f'{dev_spare}', 'contrib')]

            # Sort the subset from the top down
            pb_alloc_sub = pb_alloc_sub.sort_values('contrib', ascending=False)

            # Loop through the subset and allocate growth carrying over remainder when allocating
            alloc = proj

            for index, prop in pb_alloc_sub.iterrows():
                
                prop_dev_cur = prop[f'{dev_year}']
                prop_dev_prev = prop[f'{dev_prev}']
                prop_dev_spare = prop[f'{dev_spare}']

                if prop_dev_cur == 0.0 and prop_dev_spare:
                    
                    # Calculate the number of residential units consumed from the spare capacity
                    prop_new_dev = math.ceil(prop.contrib * proj)

                    # If the new development exceeds the spare capacity then set as the remaining spare capacity
                    if prop_new_dev > prop_dev_spare:
                        prop_new_dev = prop_dev_spare

                    # Update the development column in the subset
                    pb_alloc_sub.at[index, f'{dev_year}'] = prop_new_dev + prop_dev_prev

                    # Update the spare capacity column in the subset
                    pb_alloc_sub.at[index, f'{dev_spare}'] = prop_dev_spare - prop_new_dev

                    # Remove new development from the overall allocation
                    alloc -= prop_new_dev

                if alloc <= 0:

                    break

            # Drop extraneous columns from the subset
            pb_alloc_sub = pb_alloc_sub.drop(['contrib', f'{dev_prev}'], axis=1).set_index('pbno')

            # Update the master dataframe with the updated columns
            pb_alloc_res.update(pb_alloc_sub)

        # Set remaining null values in projection year to be that of year before (for 2031 onwards)
        if year >= 2031:
            pb_alloc_res[f'{dev_year}'] = pb_alloc_res.apply(lambda row: SetYear(row, col, year, years[i - 1]), axis=1)

# ----------------------------------

# Drop extraneous
pb_alloc_res = pb_alloc_res.drop(['contrib'], axis=1)

# Inspect
pb_alloc_res.sort_values('att_dwl_cap', ascending=False).head()

,sa2_name,det_dwl_2021,det_dwl_2026,det_dwl_2031,det_dwl_2036,det_dwl_2041,det_dwl_2046,det_dwl_2051,det_dwl_spare,det_dwl_cap,att_dwl_2021,att_dwl_2026,att_dwl_2031,att_dwl_2036,att_dwl_2041,att_dwl_2046,att_dwl_2051,att_dwl_spare,att_dwl_cap
pbno,,,,,,,,,,,,,,,,,,,
8029483,Cleveland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,295.0,765.0,1289.0,1684.0,1684.0,1684.0,0.0,1684.0
8007938,Capalaba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,57.0,62.0,67.0,73.0,80.0,200.0,280.0
8048032,Thorneside,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,273.0,273.0
8001126,Cleveland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,123.0,207.0,269.0,269.0,269.0,0.0,269.0
8059269,Capalaba,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,19.0,49.0,54.0,59.0,65.0,71.0,171.0,242.0


In [21]:
''' 
----------------------------------
Get the residential growth summary by SA2
----------------------------------
'''
# Groupby and sum
sa2_res_alloc = pb_alloc_res.reset_index(drop=True).groupby('sa2_name').agg('sum')

# Inspect
sa2_res_alloc

,det_dwl_2021,det_dwl_2026,det_dwl_2031,det_dwl_2036,det_dwl_2041,det_dwl_2046,det_dwl_2051,det_dwl_spare,det_dwl_cap,att_dwl_2021,att_dwl_2026,att_dwl_2031,att_dwl_2036,att_dwl_2041,att_dwl_2046,att_dwl_2051,att_dwl_spare,att_dwl_cap
sa2_name,,,,,,,,,,,,,,,,,,
Alexandra Hills,5720.0,5736.0,5774.0,5774.0,5774.0,5774.0,5774.0,0.0,54.0,597.0,668.0,668.0,668.0,668.0,668.0,668.0,595.0,666.0
Birkdale,4837.0,4886.0,4976.0,5123.0,5271.0,5303.0,5331.0,0.0,494.0,779.0,856.0,931.0,961.0,991.0,1004.0,1017.0,660.0,898.0
Capalaba,5478.0,5504.0,5601.0,5689.0,5689.0,5689.0,5689.0,0.0,211.0,1628.0,2013.0,2423.0,2473.0,2523.0,2582.0,2642.0,2674.0,3734.0
Cleveland,4260.0,4297.0,4401.0,4402.0,4403.0,4430.0,4458.0,11.0,209.0,2939.0,4444.0,5879.0,7030.0,8185.0,8598.0,9031.0,553.0,6670.0
Ormiston,1841.0,1900.0,1981.0,2058.0,2058.0,2058.0,2058.0,0.0,217.0,758.0,801.0,867.0,868.0,869.0,878.0,887.0,108.0,268.0
Redland Bay,6084.0,6380.0,7111.0,7790.0,8667.0,8839.0,9025.0,2720.0,5661.0,445.0,602.0,713.0,739.0,767.0,795.0,824.0,745.0,1124.0
Redland Islands,6632.0,7227.0,7864.0,8337.0,8475.0,8475.0,8475.0,0.0,1843.0,725.0,728.0,728.0,728.0,728.0,728.0,728.0,404.0,410.0
Sheldon - Mount Cotton,2683.0,2690.0,2722.0,2724.0,2724.0,2724.0,2724.0,0.0,41.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0,0.0,0.0
Thorneside,992.0,994.0,998.0,999.0,1000.0,1009.0,1018.0,17.0,43.0,597.0,596.0,596.0,596.0,596.0,596.0,596.0,349.0,385.0


# *Outputs*

In [22]:
# Output SA2 metrics to csv
sa2_res_alloc.to_csv(os.path.join(outputs_path, 'OUT_Redland_SA2_ResGrowthAllocation.csv'))

In [23]:
# Merge with spatial
pb_alloc_res_out = pd.merge(pb_curdev_lyr.loc[:, ('pbno', 'SHAPE')], pb_alloc_res.reset_index(), on='pbno', how='left')

# Output into geodatabase
pb_alloc_res_out.spatial.to_featureclass(os.path.join(alloc_gdb_path, 'OUT_FC_PropertyBase_ResGrowthAllocation'), overwrite=True)

'G:\\Shared drives\\PIESolutions_03_Projects\\J000111 - Redlands planning assumption update\\06_Working Documents\\00_GIS Directory\\00_Data\\gam_model_run\\redland_gam_alloc.gdb\\OUT_FC_PropertyBase_ResGrowthAllocation'